In [1]:
# Reproduction script to calculate exposure .csv 
# that was used in the main file for figures
# ------------------------------------------------


import warnings
warnings.simplefilter(action='ignore')
import os 
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from shapely.wkb import loads
from datetime import timedelta
from shapely.geometry import Polygon
import seaborn as sns
import scipy
from scipy.stats import ttest_ind
from shapely import wkt
import matplotlib.colors as mcolors
from datetime import datetime
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path
from typing import List, Tuple
from sklearn.cluster import DBSCAN
import multiprocessing as mp
from datetime import datetime, timedelta


In [2]:
# ---------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)

DATA_DIR  = Path("burada")
PAPER_DIR = Path("paper")

ZIP_SHP         = DATA_DIR / "zip codes" / "Export_Output_5.shp"
MONITOR_CSV     = DATA_DIR / "monitorloc.csv"
ID_MAP_CSV      = DATA_DIR / "ids.csv"
WEIGHTS_CSV     = PAPER_DIR / "bws_user_weights.csv"
QUALTRICS_CSV   = PAPER_DIR / "qualtrics.csv"
AQ_DATA_CSV     = Path("aq_data.csv")
EPA_DAILY_CSV   = Path("daily_88101_2023.csv")

CRS_WGS84 = "EPSG:4326"

# ---------------------------------------------------------------------
# 1. Zip-code polygons
# ---------------------------------------------------------------------
zip_gdf = (
    gpd.read_file(ZIP_SHP)
       .set_crs(CRS_WGS84, allow_override=True)
)

# ---------------------------------------------------------------------
# 2. Monitor locations 
# ---------------------------------------------------------------------
monitor_df  = pd.read_csv(MONITOR_CSV)
monitor_gdf = gpd.GeoDataFrame(
    monitor_df,
    geometry=gpd.points_from_xy(monitor_df["lon"], monitor_df["lat"]),
    crs=CRS_WGS84,
)

# Clip monitors to zip-code extent
monitor_gdf = gpd.clip(monitor_gdf, zip_gdf)

# ---------------------------------------------------------------------
# 3. ID mapping
# ---------------------------------------------------------------------
id_map = (
    pd.read_csv(ID_MAP_CSV)
      .rename(columns={"ids1": "id"})[["id", "ids2"]]
)

# ---------------------------------------------------------------------
# 4. User weights + demographics
# ---------------------------------------------------------------------
weights = (
    pd.read_csv(WEIGHTS_CSV)
      .rename(columns={"mt_id": "user_id"})
)

qualtrics = (
    pd.read_csv(QUALTRICS_CSV)
      .fillna("nan")
      .merge(id_map, how="left")
      .rename(columns={"ids2": "user_id"})
)

weights = weights.merge(qualtrics[["user_id", "occup"]], how="left")

# ---------------------------------------------------------------------
# 5. Air-quality datasets
# ---------------------------------------------------------------------
aq_data   = pd.read_csv(AQ_DATA_CSV)
epa_daily = pd.read_csv(EPA_DAILY_CSV)

In [3]:
# ---------------------------------------------------------------------
# Constants & paths
# ---------------------------------------------------------------------
DST_SWITCH_TS = pd.Timestamp("2023-11-05 07:00:00")  # end of DST in 2023
DATA_DIR      = Path("paper")

STAYS_CSV   = DATA_DIR / "storylines_MAPC.csv"

# ---------------------------------------------------------------------
# 1. Load stay-segments
# ---------------------------------------------------------------------
stays = pd.read_csv(
    STAYS_CSV,
    parse_dates=["started_at", "finished_at"],   
).sort_values("started_at")

# ---------------------------------------------------------------------
# 2. Correct timestamps for DST gap (pre- vs. post-07:00 5 Nov 2023)
# ---------------------------------------------------------------------
dst_offset_hours = np.where(
    stays["started_at"] < DST_SWITCH_TS, 4, 5   # 4 h before switch, 5 h after
)

stays["corrected_time_start"] = (
    stays["started_at"] - pd.to_timedelta(dst_offset_hours, unit="h")
)
stays["corrected_time_end"] = (
    stays["finished_at"] - pd.to_timedelta(
        np.where(stays["started_at"] < DST_SWITCH_TS, 4, 5), unit="h"
    )
)

# Hour-of-day convenience columns
stays["hour_start"] = stays["corrected_time_start"].dt.hour
stays["hour_end"]   = stays["corrected_time_end"].dt.hour

# ---------------------------------------------------------------------
# 3. Keep users with ≥ 10 days between first & last corrected stay
# ---------------------------------------------------------------------
span_days = (
    stays.groupby("user_id")
         .agg(span=("corrected_time_end", lambda s: (s.max() - s.min()).days))
)
approved_users = span_days.query("span > 10").index
stays = stays[stays["user_id"].isin(approved_users)]

# ---------------------------------------------------------------------
# 4. Merge demographics, filter stay-only rows, and trim durations
# ---------------------------------------------------------------------
stays = (
    stays.loc[stays["type"] == "Stay"]          # keep only “Stay” rows
         .merge(qualtrics[["user_id", "occup"]], on="user_id", how="left")  # from earlier code
         .query("30 < duration_min < 300")      # duration 30–300 min
         .copy()
)

# ---------------------------------------------------------------------
# 5. Geometry handling
# ---------------------------------------------------------------------
stays["geometry"] = [
    loads(bytes.fromhex(wkb_hex)) for wkb_hex in stays["geometry"]
]
stays_gdf = gpd.GeoDataFrame(stays, geometry="geometry", crs=CRS_WGS84)

# ---------------------------------------------------------------------
# 6. Extra temporal flags
# ---------------------------------------------------------------------
stays_gdf["day_start"]   = stays_gdf["corrected_time_start"].dt.day
stays_gdf["month_start"] = stays_gdf["corrected_time_start"].dt.month
stays_gdf["comb_dm"]     = stays_gdf["month_start"].astype(str) + "/" + stays_gdf["day_start"].astype(str)

stays_gdf["is_weekend"] = np.where(
    stays_gdf["corrected_time_start"].dt.weekday >= 5, "Yes", "No"
)


In [4]:
WAYPOINT_DIR = Path("burada/waypoints_folders")
LAT_COL      = "latitude"
LON_COL      = "longitude"
USER_COL     = "user_id"
TIME_COL     = "tracked_at"

# ---------------------------------------------------------------------
# 1. Read & concatenate waypoint CSVs
# ---------------------------------------------------------------------
waypoint_dfs = [
    pd.read_csv(csv_path) for csv_path in WAYPOINT_DIR.glob("*.csv")
]

waypoints = pd.concat(waypoint_dfs, ignore_index=True)

# ---------------------------------------------------------------------
# 2. Clip to zip-code bounding box
# (assumes `zcodes` — a GeoDataFrame — is already defined)
# ---------------------------------------------------------------------
min_lon, min_lat, max_lon, max_lat = zip_gdf.total_bounds  # xmin, ymin, xmax, ymax

waypoints = waypoints.loc[
    (waypoints[LAT_COL].between(min_lat, max_lat)) &
    (waypoints[LON_COL].between(min_lon, max_lon))
]



In [ ]:
# ---------------------------------------------------------------------
# Calculate home locations
# ---------------------------------------------------------------------
import skmob
from skmob.measures.individual import home_location
traj = skmob.TrajDataFrame(waypoints,latitude=LAT_COL,longitude=LON_COL,user_id=USER_COL,datetime=TIME_COL)

homes = home_location(traj, start_night="22:00", end_night="06:00")


In [5]:
"""
Exposure-at-Stay computation
"""
homes = pd.read_csv('homes.csv')
homes = gpd.GeoDataFrame(
    homes, geometry=gpd.points_from_xy(homes.lng, homes.lat,crs = "EPSG:4269"))


# ──────────────────────────────────────────────────────────────────────
# CONSTANTS & PATHS
# ──────────────────────────────────────────────────────────────────────
CRS_MERC  = "EPSG:3857"

BUFFER_METERS = 4000  # radius for sensor buffers

# ──────────────────────────────────────────────────────────────────────
# 1.  Build buffered sensors & supporting look-ups
# ──────────────────────────────────────────────────────────────────────
def build_buffered_sensors(
    sensors: gpd.GeoDataFrame,
    stays: gpd.GeoDataFrame,
    buf_m: int = BUFFER_METERS,
) -> gpd.GeoDataFrame:

    sensors_merc = sensors.to_crs(CRS_MERC)
    buffers      = sensors_merc.buffer(buf_m).to_crs(CRS_WGS84)

    buffered = gpd.GeoDataFrame(
        sensors[["id"]].copy(),
        geometry=buffers,
        crs=CRS_WGS84,
    )

    joined = gpd.sjoin(buffered, stays, how="inner", predicate="intersects")
    return joined.reset_index(drop=True)

# ──────────────────────────────────────────────────────────────────────
# 2.  Per-stay exposure calculation
# ──────────────────────────────────────────────────────────────────────
def _calc_single_stay(
    stay_idx: int,
    stay_lookup: pd.Series,
    buffered_sensors: gpd.GeoDataFrame,
    aq_data: pd.DataFrame,
    stays: gpd.GeoDataFrame,
    buffered_homes: gpd.GeoDataFrame,
) -> Tuple[List[int], List[float], List[float], List[str]]:
  
    """
    Compute exposure metrics for one stay, referenced by *stay_idx*.
    
    """
    # -----------------------------------------------------------------
    # A.  Gather context for this stay
    # -----------------------------------------------------------------
    stay_id      = stay_lookup[stay_idx]                  
    buf_slice    = buffered_sensors.query("index == @stay_id")
    sensor_ids   = buf_slice["id_left"].tolist()
    
    user_id      = buf_slice["user_id"].iat[0]
    stay_row     = stays.loc[stays["id"] == buf_slice["id_right"].iat[0]].iloc[0]
    
    stays = stays_clipped.reset_index()
    # Home / not-home label
    label = (
        "home"
        if not buffered_homes.query("uid == @user_id").sjoin(stays[stays['index']==stay_row.name][['geometry','id']]).empty
        else "not home"
    )
    
    start_t = stay_row["corrected_time_start"]
    end_t   = stay_row["corrected_time_end"]
    
    day_mask = (aq_data["month"] == start_t.month) & (aq_data["day"] == start_t.day)
    day_df   = aq_data.loc[day_mask].copy()
    
    daily_mean = (
        day_df.groupby(["year", "month", "day"])["pms"]
        .transform("mean")
        .rename("daily_avg_pms")
    )
    day_df["pms2"] = day_df["pms"] - daily_mean
    
    # Only rows from relevant sensors
    sensor_df = day_df[day_df["ids"].isin(sensor_ids)]
    
    # -----------------------------------------------------------------
    # B.  30-minute sliding-window aggregation
    # -----------------------------------------------------------------
    idx_store, pms_raw_store, pms_det_store, label_store = [], [], [], []
    
    
    sensor_df['lags2'] = pd.to_datetime(sensor_df['lags2'])
    
    if not sensor_df.empty:
        win_end   = end_t
        win_start = end_t - timedelta(minutes=30)
    
        while win_start > start_t:
            window = sensor_df[
                (sensor_df["lags2"] > win_start) & (sensor_df["lags2"] < win_end)
            ]
    
            if not window.empty:
                agg = (
                    window[["month", "day", "hour", "minute", "pms", "pms2"]]
                    .groupby(["month", "day", "hour", "minute"])
                    .median()
                )
    
                pms_raw = agg["pms"].mean()
                pms_det = agg["pms2"].mean()
            else:
                pms_raw = pms_det = -99.0
    
            # store
            idx_store.append(stay_id)
            pms_raw_store.append(pms_raw)
            pms_det_store.append(pms_det)
            label_store.append(label)
    
            # slide 30 min earlier
            win_end   -= timedelta(minutes=30)
            win_start -= timedelta(minutes=30)
    else:
        # No sensors in vicinity
        idx_store.append(stay_id)
        pms_raw_store.append(-99.0)
        pms_det_store.append(-99.0)
        label_store.append(label)
    


    return idx_store, pms_raw_store, pms_det_store, label_store


# ──────────────────────────────────────────────────────────────────────
# 3.  Wrapper for multiprocessing
# ──────────────────────────────────────────────────────────────────────
def run_parallel_exposure(
    buffered_sensors: gpd.GeoDataFrame,
    stays: gpd.GeoDataFrame,
    df2t: pd.DataFrame,
    buffered_homes: gpd.GeoDataFrame,
    n_jobs: int | None = None,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Dispatch per-stay exposure computations in parallel
    """
    indess       = buffered_sensors["index"].unique()
    stay_lookup  = pd.Series(indess)  

    pool = mp.Pool(processes=n_jobs or mp.cpu_count())
    try:
        results = pool.starmap(
            _calc_single_stay,
            [
                (
                    idx,
                    stay_lookup,
                    buffered_sensors,
                    aq_data,
                    stays,
                    buffered_homes,
                )
                for idx in range(len(stay_lookup))
            ],
        )
    finally:
        pool.close()
        pool.join()

    # Flatten the parallel lists
    idx_all, pms_raw_all, pms_det_all, label_all = [], [], [], []
    for ids_, raw_, det_, lab_ in results:
        idx_all.extend(ids_)
        pms_raw_all.extend(raw_)
        pms_det_all.extend(det_)
        label_all.extend(lab_)

    exposures_raw = pd.DataFrame(
        {"index": idx_all, "exposure": pms_raw_all, "hm": label_all}
    )
    exposures_det = pd.DataFrame(
        {"index": idx_all, "exposure": pms_det_all, "hm": label_all}
    )

    return exposures_raw, exposures_det


In [6]:
# ---------------------------------------------------------------------
# Buffered home locations (200 m radius)
# ---------------------------------------------------------------------
BUFFER_HOME_M = 200  # metres

buffered_homes = (
    homes.to_crs(CRS_MERC)                   # project to metres
         .buffer(BUFFER_HOME_M)              # create 200 m buffers
         .to_crs(CRS_WGS84)                  # back to lat/long
         .pipe(                              # wrap into a GeoDataFrame
             lambda geom: gpd.GeoDataFrame(
                 {"uid": homes["uid"].values, "geometry": geom},
                 crs=CRS_WGS84,
             )
         )
         .reset_index(drop=True)
)


print("▶  Starting exposure calculations:", datetime.now().isoformat(" ", "seconds"))

sensors_in_bounds = monitor_gdf[monitor_gdf["id"].isin(np.unique(aq_data["ids"]))]

stays_clipped = gpd.clip(stays_gdf,zip_gdf)
stays_clipped = stays_clipped.reset_index()

buffered_sensors = (
    build_buffered_sensors(sensors_in_bounds, stays_clipped)
    .merge(weights[["user_id", "acs_weight"]], on="user_id", how="left")
)


▶  Starting exposure calculations: 2025-08-11 09:17:52


In [7]:

stays_exp, stays_exp_detrended = run_parallel_exposure(
    buffered_sensors,
    stays_clipped, 
    aq_data,
    buffered_homes,
)

print("✔  Done:", datetime.now().isoformat(" ", "seconds"))

stays_exposure2=stays_exp_detrended.merge(stays_clipped)
stays_exposure=stays_exp.merge(stays_clipped)

✔  Done: 2025-08-11 09:29:34


In [8]:
buffered_sensors = buffered_sensors[['id_left','geometry']].drop_duplicates('id_left')
buffered_sensors = gpd.sjoin(buffered_sensors, homes, how="inner")

# 4. House-keeping objects used later in the script
bufferedsensors_home = buffered_sensors.reset_index(drop=True)
un_user              = bufferedsensors_home["uid"].unique().tolist()
work_ids             = buffered_homes["uid"].unique()


In [9]:

# ────────────────────────────────────────────────────────────────
# 2. Per-stay exposure calculation (one process)
# ────────────────────────────────────────────────────────────────
def _calc_stay_exposure(
                    idx,
                    stay_lookup,
                    buffered_sensors,
                    aq_data,
                    stays,
                    buffered_homes
) -> Tuple[List[int], List[float], List[float], List[float]]:

    stay_id   = stay_lookup[idx]
    slice_    = buffered_sensors.query("index == @stay_id")
    sensor_ids = slice_["id_left"].tolist()
    uid        = slice_["user_id"].iat[0]
    
    stay_geom  = stays_clipped.loc[stays_clipped["id"] == slice_["id_right"].iat[0]]
    home_sensor_ids = buffered_homes.query("uid == @uid")["id"].unique()
    
    # Time bounds
    start_t = slice_["corrected_time_start"].iat[0]
    end_t   = slice_["corrected_time_end"].iat[0]
    
    # Extract same-day rows once
    day_mask = (aq_data["month"] == start_t.month) & (aq_data["day"] == start_t.day)
    day_df   = aq_data.loc[day_mask].copy()
    
    # Daily detrend
    day_df["pms2"] = day_df["pms"] - day_df.groupby(
        ["year", "month", "day"]
    )["pms"].transform("mean")
    
    sensor_df      = day_df[day_df["ids"].isin(sensor_ids)]
    sensor_home_df = day_df[day_df["ids"].isin(home_sensor_ids)]
    
    idxs, pms_raw, pms_det, pms_home = [], [], [], []
    
    if not sensor_df.empty:
        sensor_df['lags2'] = pd.to_datetime(sensor_df['lags2'])
        sensor_home_df['lags2'] = pd.to_datetime(sensor_home_df['lags2'])
        win_end   = end_t
        win_start = end_t - timedelta(minutes=30)
    
        while win_start > start_t:
            w = sensor_df.query("@win_start < lags2 < @win_end")
            h = sensor_home_df.query("@win_start < lags2 < @win_end")
    
            if not w.empty and not h.empty:
                agg_w = (
                    w.groupby(["month", "day", "hour", "minute"])
                     .median()[["pms", "pms2"]]
                     .mean()
                )
                agg_h = (
                    h.groupby(["month", "day", "hour", "minute"])
                     .median()["pms"]
                     .mean()
                )
                p_raw = agg_w["pms"]
                p_det = agg_w["pms2"]
                p_home = agg_h
            else:
                p_raw = p_det = p_home = -99.0
    
            idxs.append(stay_id)
            pms_raw.append(p_raw)
            pms_det.append(p_det)
            pms_home.append(p_home)
    
            win_end   -= timedelta(minutes=30)
            win_start -= timedelta(minutes=30)
    else:
        idxs.append(stay_id)
        pms_raw.append(pms_det.append(pms_home.append(-99.0)))

    return idxs, pms_raw, pms_det, pms_home


# ────────────────────────────────────────────────────────────────
# 3. Driver: parallel processing & final merge
# ────────────────────────────────────────────────────────────────
def compute_stay_exposure_home() -> gpd.GeoDataFrame:


    pool = mp.Pool(mp.cpu_count())
    try:
        results = pool.starmap(
            _calc_stay_exposure,
            [
                (
                    idx,
                    stay_lookup,
                    buffered_sensors,
                    aq_data,
                    stays_clipped,
                    buffered_homes,
                )
                for idx in range(len(stay_lookup))
            ],
        )
    finally:
        pool.close()
        pool.join()
    # d.  COLLATE
    idx_all, p_raw_all, p_det_all, p_home_all = [], [], [], []
    for idxs, raws, dets, homes in results:
        idx_all.extend(idxs)
        p_raw_all.extend(raws)
        p_det_all.extend(dets)
        p_home_all.extend(homes)

    exposure_df = pd.DataFrame(
        {"index_right": idx_all,
         "exposure":    p_raw_all,
         "home":        p_home_all}
    )

    return exposure_df



In [10]:
buffered_sensors = (
    build_buffered_sensors(sensors_in_bounds, stays_clipped)
    .merge(weights[["user_id", "acs_weight"]], on="user_id", how="left")
)

indess       = buffered_sensors["index"].unique()
stay_lookup  = pd.Series(indess)  # simple positional index ➜ stay_id map

BUFFER_HOME_M = 4000  # metres

buffered_homes = (
    homes.to_crs(CRS_MERC)                   # project to metres
         .buffer(BUFFER_HOME_M)              # create 200 m buffers
         .to_crs(CRS_WGS84)                  # back to lat/long
         .pipe(                              # wrap into a GeoDataFrame
             lambda geom: gpd.GeoDataFrame(
                 {"uid": homes["uid"].values, "geometry": geom},
                 crs=CRS_WGS84,
             )
         )
         .reset_index(drop=True)
)
buffered_homes = buffered_homes.sjoin(sensors_in_bounds[['id','geometry']])

print("✔  Start:", datetime.now().isoformat(" ", "seconds"))

stays_exposure_home = compute_stay_exposure_home()

print("✔  Done:", datetime.now().isoformat(" ", "seconds"))


✔  Start: 2025-08-11 09:29:35
✔  Done: 2025-08-11 09:40:24


In [11]:
stays_exposure_home = stays_exposure_home.merge(stays_clipped,right_on='index',left_on='index_right')